In [127]:
# Import Data
import pandas as pd
import numpy as np

file_path = "/Users/samhenderson/Downloads/sample_datasets.xlsx"
xls = pd.ExcelFile(file_path)

applications= pd.read_excel(xls, "applications")
customers= pd.read_excel(xls, "customers")
stores= pd.read_excel(xls, "stores")
marketing= pd.read_excel(xls, "marketing")

In [128]:
print("applications:", applications.shape)
print("customers:", customers.shape)
print("stores:", stores.shape)
print("marketing:", marketing.shape)

applications: (65535, 10)
customers: (65535, 11)
stores: (250, 6)
marketing: (16, 6)


In [129]:
# Clean Data
marketing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  16 non-null     int64 
 1   id          16 non-null     int64 
 2   name        16 non-null     object
 3   spend       16 non-null     int64 
 4   start_date  15 non-null     object
 5   end_date    15 non-null     object
dtypes: int64(3), object(3)
memory usage: 896.0+ bytes


In [130]:
# Update columns to correct data type
applications["submit_date"] = pd.to_datetime(applications["submit_date"])
customers["DOB"] = pd.to_datetime(customers["DOB"])
stores["start_dt"] = pd.to_datetime(stores["start_dt"])
marketing["start_date"] = pd.to_datetime(marketing["start_date"])
marketing["end_date"] = pd.to_datetime(marketing["end_date"])

In [131]:
# Standardize store identifier by removing prefix
applications["store"] = applications["store"].str.replace("store_", "", regex=False)
stores["store"] = stores["store"].str.replace("store_", "", regex=False)

In [132]:
marketing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  16 non-null     int64         
 1   id          16 non-null     int64         
 2   name        16 non-null     object        
 3   spend       16 non-null     int64         
 4   start_date  15 non-null     datetime64[ns]
 5   end_date    15 non-null     datetime64[ns]
dtypes: datetime64[ns](2), int64(3), object(1)
memory usage: 896.0+ bytes


In [133]:
marketing.head()

,Unnamed: 0,id,name,spend,start_date,end_date
0,0,0,No Campaign,0,NaT,NaT
1,1,1,Google Ads,4798711,2022-01-26,2027-07-03
2,2,2,Facebook Ads,704391,2023-09-04,2029-12-28
3,3,3,Instagram Ads,1042189,2023-01-22,2029-05-23
4,4,4,Twitter Ads,3996752,2023-05-17,2028-05-08


In [134]:
# Drop unnecessary columns from each sheet
applications = applications.drop(columns=["Unnamed: 0"])
customers = customers.drop(columns=["Unnamed: 0"])
stores = stores.drop(columns=["Unnamed: 0"])
marketing = marketing.drop(columns=["Unnamed: 0"])

In [135]:
customers.head()

,customer_id,DOB,first_name,last_name,email,phone_number,language,income,title,campaign
0,755880622768,1997-11-11,Douglas,Ramirez,vgrant@example.net,001-304-226-7086x106,Spanish,13083,Ambulance person,13
1,332145121573,1997-11-30,Mary,Cruz,tyler38@example.com,270-279-7276,English,13143,IT sales professional,0
2,144890296478,1997-12-22,Bryan,Anderson,stevengreene@example.org,(828)977-0809,English,7522,Licensed conveyancer,8
3,419615144299,2005-05-18,Christine,Murray,christinesummers@example.org,218-169-1692x1171,Spanish,8963,Archivist,13
4,848253698374,2008-03-14,Jared,Mayer,suttonjessica@example.com,669-062-7993x616,English,2771,Social researcher,0


In [136]:
# Verify that there are no null fields
applications.isna().sum()

application_id         0
customer_id            0
store                  0
submit_date            0
approved               0
approved_date      32779
approved_amount    32779
dollars_used       49261
lease_grade            0
dtype: int64

In [137]:
# Task 1 import and connect to SQLite3
import sqlite3
conn = sqlite3.connect(":memory:")
applications.to_sql("applications", conn, index=False)

In [142]:
# Using SQL to group Total_Applications, Approved_Application, and Funded_Leases by month
query_task1 = """

SELECT
    strftime('%Y-%m', submit_date) AS year_month,
    COUNT(*) AS Total_Applications,
    SUM(approved) AS Approved_Applications,
    SUM(CASE WHEN dollars_used > 0 THEN 1 ELSE 0 END) AS Used_Applications 
FROM applications
GROUP BY strftime('%Y-%m', submit_date)
ORDER BY year_month
"""

In [143]:
task1 = pd.read_sql_query(query_task1, conn)
task1.head(10)


,year_month,Total_Applications,Approved_Applications,Used_Applications
0,2022-01,2775,1383,660
1,2022-02,2514,1282,628
2,2022-03,2723,1352,686
3,2022-04,2693,1284,649
4,2022-05,2787,1346,672
5,2022-06,2747,1391,712
6,2022-07,2824,1376,678
7,2022-08,2744,1379,683
8,2022-09,2706,1350,664
9,2022-10,2818,1422,694


In [144]:
task1.to_csv("task1.csv", index=False)

In [149]:
# Task 2 Using SQL to compare Approved to Used

query_task2 = """

SELECT
    strftime('%Y-%m', submit_date) AS year_month,
    AVG(CASE
    WHEN approved = 1 THEN approved_amount
    END) AS Avg_Approved,
    AVG(CASE
    WHEN dollars_used > 0 THEN dollars_used
    END) AS Avg_Used,
    CASE
        WHEN SUM(CASE WHEN approved = 1 AND dollars_used > 0 THEN approved_amount ELSE 0 END) > 0
        THEN
            1.0 * SUM(CASE WHEN approved = 1 AND dollars_used > 0 THEN dollars_used ELSE 0 END)
            / SUM(CASE WHEN approved = 1 AND dollars_used > 0 THEN approved_amount ELSE 0 END)
    END AS Pct_Approved_Dollars_Used

FROM applications
GROUP BY strftime('%Y-%m', submit_date)
ORDER BY year_month

"""

In [150]:
task2 = pd.read_sql_query(query_task2, conn)
# Fix formatting
task2["Pct_Approved_Dollars_Used"] = (task2["Pct_Approved_Dollars_Used"]*100).round(1)
task2["Avg_Approved"] = task2["Avg_Approved"].round()
task2["Avg_Used"] = task2["Avg_Used"].round()
task2.head(10)

,year_month,Avg_Approved,Avg_Used,Pct_Approved_Dollars_Used
0,2022-01,3122.0,1634.0,52.6
1,2022-02,3155.0,1585.0,50.3
2,2022-03,3112.0,1610.0,50.9
3,2022-04,3099.0,1611.0,52.5
4,2022-05,3052.0,1626.0,54.1
5,2022-06,3125.0,1597.0,51.9
6,2022-07,3164.0,1633.0,51.5
7,2022-08,3129.0,1581.0,50.4
8,2022-09,3125.0,1650.0,52.9
9,2022-10,3148.0,1631.0,51.0


In [151]:
task2.to_csv("task2.csv", index=False)

In [152]:
# Task 3

applications["used_flag"] = applications["dollars_used"] > 0

# Aggregate by store
task3 = (
    applications
    .groupby("store", as_index=False)
    .agg(
        Applications=("application_id", "count"),
        Approved=("approved", "sum"),
        Used=("used_flag", "sum"),
        Total_Approved_Amount=("approved_amount", "sum"),
        Total_Used_Amount=("dollars_used", "sum"),))

# Find rates
task3["Approved %"] = np.where(
    task3["Applications"] > 0,
    task3["Approved"] / task3["Applications"],
    np.nan)

task3["Used %"] = np.where(
    task3["Approved"] > 0,
    task3["Used"] / task3["Approved"],
    np.nan)

# Reorder columns
task3 = task3[
    [   "store",
        "Applications",
        "Approved",
        "Approved %",
        "Used",
        "Used %",
        "Total_Approved_Amount",
        "Total_Used_Amount",]]

task3 = task3.sort_values(by="Total_Used_Amount",ascending=False)

task3["Approved %"] = (task3["Approved %"]*100).round(1)
task3["Used %"] = (task3["Used %"]*100).round(1)

task3.head(10)

,store,Applications,Approved,Approved %,Used,Used %,Total_Approved_Amount,Total_Used_Amount
165,247,327,175,53.5,101,57.7,543201.0,157407.0
10,107,265,142,53.6,81,57.0,426954.0,148079.0
41,135,284,142,50.0,77,54.2,437801.0,142274.0
197,51,304,160,52.6,89,55.6,487542.0,142057.0
137,221,260,130,50.0,75,57.7,402020.0,137689.0
35,13,278,145,52.2,84,57.9,456719.0,135526.0
117,203,282,149,52.8,90,60.4,457473.0,135373.0
83,173,278,153,55.0,75,49.0,487954.0,133215.0
81,171,287,153,53.3,81,52.9,486626.0,132396.0
212,65,292,145,49.7,80,55.2,446946.0,132227.0


In [153]:
# Task 4
apps_with_campaign = applications.merge(
    customers[["customer_id", "campaign"]],
    on="customer_id",
    how="left")

# Remove "no campaign" rows (campaign == 0) and any nulls just in case
apps_with_campaign = apps_with_campaign[
    apps_with_campaign["campaign"].notna() & (apps_with_campaign["campaign"] != 0)]

In [154]:
# Sum used dollars by campaign
used_by_campaign = (
    apps_with_campaign
    .groupby("campaign", as_index=False)
    .agg(total_used_dollars=("dollars_used", "sum")))

# Join to marketing spend and campaign names
task4 = used_by_campaign.merge(
    marketing[["id", "name", "spend"]],
    left_on="campaign",
    right_on="id",
    how="left")

# Format and sort results for comparison
task4 = task4[["name", "spend", "total_used_dollars"]].rename(columns={
    "name": "Marketing Name",
    "spend": "Marketing Spend",
    "total_used_dollars": "Total Used Dollars"
}).sort_values("Total Used Dollars", ascending=False)

task4.head(14)


,Marketing Name,Marketing Spend,Total Used Dollars
9,Trade Shows,3344380,1281597.0
0,Google Ads,4798711,1247539.0
3,Twitter Ads,3996752,1226074.0
6,Content Marketing,2728930,1202427.0
13,TV Ads,2539860,1201906.0
10,Referral Marketing,394522,1201689.0
1,Facebook Ads,704391,1199160.0
4,LinkedIn Ads,1404633,1196927.0
12,Affiliate Marketing,4708501,1174320.0
2,Instagram Ads,1042189,1170408.0


In [155]:
task4 = task4.sort_values("Marketing Spend", ascending=True)
task4.to_csv("task4.csv", index=False)

In [161]:
# Task 5

# Aggregate approved applications and actual usage by store
task5 = (
    applications
    .groupby("store", as_index=False)
    .agg(
        Approved=("approved", "sum"),
        Used=("used_flag", "sum")))

In [163]:
task5["Used %"] = task5["Used"] / task5["Approved"]
task5["Used %"] = (task5["Used %"]*100).round(1)

In [164]:
task5 = task5.sort_values("Used %", ascending=False)
task5.head(10)

,store,Approved,Used,Used %
117,203,149,90,60.4
126,211,119,71,59.7
206,6,113,66,58.4
182,38,122,71,58.2
193,48,129,75,58.1
35,13,145,84,57.9
137,221,130,75,57.7
165,247,175,101,57.7
31,126,129,74,57.4
10,107,142,81,57.0


In [160]:
task5.to_csv("task5.csv", index=False)